In [ ]:
import ccxt
import pandas as pd
import time
from datetime import datetime

# =========================
# Market Data (LIVE)
# =========================

class MarketData:
    def __init__(self, exchange, symbol, timeframe="1s", limit=50):
        self.exchange = exchange
        self.symbol = symbol
        self.timeframe = timeframe
        self.limit = limit

    def fetch(self):
        ohlcv = self.exchange.fetch_ohlcv(
            self.symbol,
            timeframe=self.timeframe,
            limit=self.limit
        )
        return pd.DataFrame(
            ohlcv,
            columns=["timestamp", "open", "high", "low", "close", "volume"]
        )

# =========================
# Strategy (Simple SMA)
# =========================

class Strategy:
    def __init__(self, fast=5, slow=7):
        self.fast = fast
        self.slow = slow

    def generate_signal(self, data):
        if len(data) < self.slow:
            return "HOLD"

        fast_sma = data["close"].rolling(self.fast).mean().iloc[-1]
        slow_sma = data["close"].rolling(self.slow).mean().iloc[-1]

        if fast_sma > slow_sma:
            return "BUY"
        elif fast_sma < slow_sma:
            return "SELL"
        return "HOLD"

# =========================
# Futures Position
# =========================

class Position:
    def __init__(self, side, entry_price, position_size, leverage):
        self.side = side
        self.entry_price = entry_price
        self.position_size = position_size
        self.leverage = leverage

    def unrealized_pnl(self, price):
        if self.side == "LONG":
            return (price - self.entry_price) * self.position_size * self.leverage
        else:
            return (self.entry_price - price) * self.position_size * self.leverage

    def liquidation_price(self):
        if self.side == "LONG":
            return self.entry_price * (1 - 1 / self.leverage)
        else:
            return self.entry_price * (1 + 1 / self.leverage)

# =========================
# Account (Risk + Fees)
# =========================

class Account:
    def __init__(self, balance, risk_per_trade=0.01, fee_rate=0.0004):
        self.balance = balance
        self.equity = balance
        self.risk_per_trade = risk_per_trade
        self.fee_rate = fee_rate
        self.position = None

        # stats
        self.trade_log = []

    def calculate_position_size(self, price, leverage):
        risk_amount = self.balance * self.risk_per_trade
        return risk_amount / price

    def open_position(self, side, price, leverage):
        size = self.calculate_position_size(price, leverage)
        fee = size * price * self.fee_rate

        self.balance -= fee
        self.position = Position(side, price, size, leverage)

        print(f"[OPEN {side}] @ {price:.2f} | Size: {size:.4f}")

    def close_position(self, price):
        pnl = self.position.unrealized_pnl(price)
        fee = self.position.position_size * price * self.fee_rate

        self.balance += pnl - fee

        self.trade_log.append(pnl)

        print(
            f"[CLOSE] @ {price:.2f} | "
            f"PnL: {pnl:.2f} | Balance: {self.balance:.2f}"
        )

        self.position = None

# =========================
# Paper Trading Engine
# =========================

class PaperFuturesEngine:
    def __init__(self, market, strategy, account, leverage=5):
        self.market = market
        self.strategy = strategy
        self.account = account
        self.leverage = leverage
        self.last_timestamp = None

    def run(self):
        print("🚀 PAPER FUTURES BOT STARTED\n")

        while True:
            try:
                data = self.market.fetch()
                latest = data.iloc[-1]
                ts = latest["timestamp"]

                # only act on NEW candle
                if ts == self.last_timestamp:
                    time.sleep(2)
                    continue

                self.last_timestamp = ts
                price = latest["close"]

                # update equity
                if self.account.position:
                    self.account.equity = (
                        self.account.balance
                        + self.account.position.unrealized_pnl(price)
                    )

                    # liquidation check
                    liq = self.account.position.liquidation_price()
                    if (
                        self.account.position.side == "LONG" and price <= liq
                    ):
                        print("💀 LIQUIDATED")
                        break

                signal = self.strategy.generate_signal(data)

                if signal == "BUY" and self.account.position is None:
                    self.account.open_position("LONG", price, self.leverage)

                elif signal == "SELL" and self.account.position:
                    self.account.close_position(price)

                print(
                    f"[{datetime.utcnow().strftime('%H:%M:%S')}] "
                    f"Price: {price:.2f} | "
                    f"Equity: {self.account.equity:.2f}"
                )

                time.sleep(2)

            except KeyboardInterrupt:
                print("\n🛑 Bot stopped manually")
                break


In [ ]:
exchange = ccxt.bitget({
    "options": {"defaultType": "future"}
})

market = MarketData(exchange, "TON/USDT", timeframe="1m")
strategy = Strategy(2,3)
account = Account(balance=1000)

engine = PaperFuturesEngine(market, strategy, account)
engine.run()

🚀 PAPER FUTURES BOT STARTED

[OPEN LONG] @ 1.49 | Size: 6.7159
[16:46:33] Price: 1.49 | Equity: 1000.00


/tmp/ipython-input-290599340.py:169: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f"[{datetime.utcnow().strftime('%H:%M:%S')}] "


[CLOSE] @ 1.49 | PnL: -0.03 | Balance: 999.96
[16:47:02] Price: 1.49 | Equity: 999.96


In [ ]:
import numpy as np

trades = account.trade_log

if len(trades) == 0:
    print("No trades executed.")
else:
    total_trades = len(trades)
    wins = len([p for p in trades if p > 0])
    losses = total_trades - wins

    win_rate = wins / total_trades * 100
    total_pnl = sum(trades)
    avg_pnl = np.mean(trades)
    max_win = max(trades)
    max_loss = min(trades)

    print("📊 TRADE STATISTICS\n")
    print(f"Total Trades : {total_trades}")
    print(f"Win Rate     : {win_rate:.2f}%")
    print(f"Total PnL    : {total_pnl:.2f}")
    print(f"Avg PnL      : {avg_pnl:.2f}")
    print(f"Max Win      : {max_win:.2f}")
    print(f"Max Loss     : {max_loss:.2f}")
    print(f"Final Equity : {account.balance:.2f}")
